In [ ]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
from torch.utils.data import DataLoader, Dataset
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import precision_score, recall_score, accuracy_score
from tqdm import tqdm
import matplotlib.pyplot as plt
import time
import psutil
import os

In [ ]:
# Constants
INPUT_DIM = 768
HIDDEN_DIM = 256
OUTPUT_DIM = 33
NUM_LAYERS = 2
BATCH_SIZE = 64
LEARNING_RATE = 0.001
NUM_EPOCHS = 50
DROPOUT = 0.5

In [ ]:
# Check for GPU availability and setup
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

# Load data
train_data = pd.read_hdf('train_embeddings.h5')
test_data = pd.read_hdf('test_embeddings.h5')
val_data = pd.read_hdf('val_embeddings.h5')

# Encode labels
label_encoder = LabelEncoder()
train_data['source'] = label_encoder.fit_transform(train_data['source'])
test_data['source'] = label_encoder.transform(test_data['source'])
val_data['source'] = label_encoder.transform(val_data['source'])

# Dataset class
class EmbeddingDataset(Dataset):
    def __init__(self, embeddings, labels):
        self.embeddings = np.array(embeddings)
        self.labels = np.array(labels)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.embeddings[idx], self.labels[idx]

In [ ]:
# Data loaders
train_dataset = EmbeddingDataset(train_data['gpt2_embeddings'], train_data['source'])
test_dataset = EmbeddingDataset(test_data['gpt2_embeddings'], test_data['source'])
val_dataset = EmbeddingDataset(val_data['gpt2_embeddings'], val_data['source'])

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [ ]:
# Model definition
class ComplexLSTM(nn.Module):
    def __init__(self):
        super(ComplexLSTM, self).__init__()
        self.lstm = nn.LSTM(INPUT_DIM, HIDDEN_DIM, NUM_LAYERS, dropout=DROPOUT, batch_first=True)
        self.fc = nn.Linear(HIDDEN_DIM, OUTPUT_DIM)

    def forward(self, x):
        self.lstm.flatten_parameters()
        _, (hn, _) = self.lstm(x)
        out = self.fc(hn[-1])
        return out

# Setup model, loss, and optimizer
model = ComplexLSTM().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [ ]:
# Function to track memory and GPU usage
def track_resources():
    gpu_memory = torch.cuda.memory_allocated(device) if torch.cuda.is_available() else 0
    memory = psutil.virtual_memory().used
    cpu = psutil.cpu_percent()
    return gpu_memory, memory, cpu

# Training with metrics
def train_model(loader):
    model.train()
    start_time = time.time()
    resources = []
    for epoch in range(NUM_EPOCHS):
        loop = tqdm(loader, leave=True)
        for embeddings, labels in loop:
            embeddings = torch.tensor(embeddings, dtype=torch.float32).to(device)
            labels = torch.tensor(labels, dtype=torch.long).to(device)
            optimizer.zero_grad()
            outputs = model(embeddings)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            resources.append(track_resources())
            loop.set_description(f'Epoch {epoch+1}')
            loop.set_postfix(loss=loss.item())
    training_time = time.time() - start_time
    return training_time, resources

# Evaluation function
def evaluate_model(loader):
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for embeddings, labels in loader:
            embeddings = torch.tensor(embeddings, dtype=torch.float32).to(device)
            labels = torch.tensor(labels, dtype=torch.long).to(device)
            outputs = model(embeddings)
            _, predicted = torch.max(outputs.data, 1)
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds, average='macro')
    recall = recall_score(all_labels, all_preds, average='macro')
    return accuracy, precision, recall

In [ ]:
# Execute training and evaluation
training_time, resources = train_model(train_loader)
accuracy, precision, recall = evaluate_model(val_loader)

# Plot resources
times = [r[0] for r in resources]
memory_usage = [r[1] for r in resources]
cpu_usage = [r[2] for r in resources]

plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(memory_usage, label='Memory Usage')
plt.title('Memory Usage Over Time')
plt.xlabel('Iterations')
plt.ylabel('Memory (Bytes)')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(cpu_usage, label='CPU Usage')
plt.title('CPU Usage Over Time')
plt.xlabel('Iterations')
plt.ylabel('Percentage')
plt.legend()
plt.show()

# Display metrics
print(f'Training Time: {training_time} seconds')
print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')